In [1]:
# Imports
import numpy as np
import random

In [204]:
# Parameters
genome_size = 16
max_group_size = 6
starting_group_size = 5
group_number = 4
mutation_rate = 0.5
group_split_rate = 0.5

In [417]:
# Create initial population
initial_population = np.full((group_number, max_group_size, genome_size), -2, dtype=np.int8)
initial_particles = np.random.randint(0, 2, size=(group_number, starting_group_size, genome_size), dtype=np.int8)
initial_population[:, :starting_group_size, :] = initial_particles
population = initial_population.copy()

In [4]:
# Fitness table?

# Take population array and calculate the fitness for each indiviual in the initial population
# If a group splits or a individual is born, recalculate the fitnesses in that/those group(s)
# Than update this table 
# To not calculate the fitnesses of each individuals each timestep but only SOME individuals SOMEtimes

In [ ]:
# Function for when a group is full
def groupFullEvent(pop_input, group_input):
    # Either one dies or group splits

    # If one dies 
    if random.random() < group_split_rate:
        # Select unlucky member
        unlucky_member = np.random.choice([group_input])
        # Set slot to 'empty'
        pop_input[group_input, unlucky_member, :] = -2
        print(f"Individual {unlucky_member} in group {group_input} has died.")
        return pop_input
    
    # If group splits:
    # Determine group that gets removed
    else:
        # All groups except the one that just split
        left_over_groups = [i for i in range(group_number) if i != group_input]
        unlucky_group = np.random.choice(left_over_groups)
        # Boolean mask to split the group in a random way
        move_mask = np.random.choice([True, False], size = max_group_size, p = [0.5, 0.5])
        
        if not np.any(move_mask): # If all false, select 1 random to be true
            move_mask[np.random.randint(0, max_group_size)] = True
        elif np.all(move_mask):   # Vice versa
            move_mask[np.random.randint(0, max_group_size)] = False
            
        # Move those marked 'True' to new group
        pop_input[unlucky_group, move_mask, :] = pop_input[group_input, move_mask, :].copy()
        
        # Set the rest to -2 ('empty')
        pop_input[unlucky_group, ~move_mask, :] = -2
        
        # Set those who moved to -2 in original group
        pop_input[group_input, move_mask, :] = -2

        # ALSO RECALCULATE FITNESS
        
        # Debug
        moved_count = np.sum(move_mask)
        print(f"Group {group_input} split and replaced group {unlucky_group}.")
        print(f"{moved_count} individuals moved. {max_group_size - moved_count} stayed.")
        
        return pop_input

In [ ]:
# Function for reproduction

def reproductionEvent(pop_input):
    # List valid genomes (all non-empty ones)
    valid_genomes = [
    (i, ind) 
    for i, group in enumerate(population) 
    for ind in group 
    if ind[0] != -2
    ]
    
    # Select random individual
    group_id, random_member = random.choice(valid_genomes)

    print(f"Selected genome: {random_member}")
    print(f"Belongs to group: {group_id}")

    # Copy genome for reproduction
    genome_copy = random_member.copy()

    # Mutation with certain mutation rate (added later)
    # if random.random() < mutation_rate: 
    #   Flip one random locus
    
    # Get empty spots in its own group to be placed into
    group_members = pop_input[group_id]
    empty_slots = np.where(group_members[:, 0] == -2)[0]
    
    # Get a empty slot (jsut take the first one)
    first_empty_slot = empty_slots[0]
    # Set the empty slot to the copy genome
    pop_input[group_id, first_empty_slot] = genome_copy
    print(f"Reproduced member in group {group_id} at slot {first_empty_slot}")
    # Trigger groupFUllEvent if the group is full (if the last empty slot has been used)
    if len(empty_slots) == 1:
        print(f"This group ({group_id}) is now full.")
        groupFullEvent(pop_input, group_id)


    # If mutated, recalculate fitness. Otherwise just copy fitness

    return pop_input

In [447]:
reproductionEvent(population)


Selected genome: [1 0 1 0 1 0 1 1 0 0 1 1 0 1 0 1]
Belongs to group: 1
Reproduced member in group 1 at slot 1


array([[[-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2],
        [ 0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0],
        [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2],
        [ 1,  0,  1,  0,  1,  0,  1,  1,  0,  0,  1,  1,  0,  1,  0,  1],
        [ 0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0],
        [ 0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0]],

       [[ 1,  0,  1,  0,  1,  0,  1,  1,  0,  0,  1,  1,  0,  1,  0,  1],
        [ 1,  0,  1,  0,  1,  0,  1,  1,  0,  0,  1,  1,  0,  1,  0,  1],
        [ 1,  0,  1,  0,  1,  0,  1,  1,  0,  0,  1,  1,  0,  1,  0,  1],
        [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2],
        [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2],
        [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2]],

       [[ 0,  1,  1,  0,  1,  0,  1,  0,  0,  1,  0,  1,  0,  0,  1,  1],
        [ 0,  1,  1,  0,  1,  0,  